In [1]:
%load_ext autoreload
%autoreload 2

from domino.slices import collect_settings
from domino.slices.celeba import CelebASliceBuilder
from domino.evaluate import run_sdm, score_sdms
from domino.train import train_settings
import numpy as np
import terra

/home/sabri/code/meerkat/meerkat/nn/__init__.py:7: ExperimentalWarning: The `meerkat.nn` module is experimental and has limited test coverage. Proceed with caution.
  warnings.warn(


In [2]:
from domino.emb.clip import get_wiki_words
#dp = get_wiki_words(top_k=25_000, eng_only=True).load()
word_dp = get_wiki_words.out().load()

In [42]:
from domino.emb.clip import generate_phrases, CELEBA_PHRASE_TEMPLATES
phrase_dp = generate_phrases(word_dp[:100], templates=CELEBA_PHRASE_TEMPLATES, num_candidates=1_00)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor(13.4990, device='cuda:0')



ValueError: Cannot create column out of data of type <class 'numpy.float32'>

In [36]:
phrase_dp.sort_values("prob", ascending=True)

,prob,text
698,24.308901,a photo of a person with the same name.
420,24.415701,a photo of a person sitting next to him.
510,24.538908,a photo of a person standing next to him.
305,26.439959,a photo of a person leaning against a wall.
4877,26.592392,a photo of a person sitting at a table.
...,...,...
10808,1789.102661,a photo of a person river had knew.
7817,1883.718872,a photo of a person in or player ).
7710,2099.983887,"a photo of a person.. director ""."
13046,2396.800293,a photo of a person river helped met.


In [31]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np 
from tqdm import tqdm

gpt_model = GPT2LMHeadModel.from_pretrained('gpt2').to(0)
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

scores = []
for text in tqdm(candidate_sents["output_phrase"]):
    tokens_tensor = gpt_tokenizer.encode( text, add_special_tokens=False, return_tensors="pt").to(0) 
    loss=gpt_model(tokens_tensor, labels=tokens_tensor)[0]
    scores.append( 
        {"prob": np.exp(loss.cpu().detach().numpy()), "text": text}       
    )
dp = pd.DataFrame(scores)

100%|██████████| 1000/1000 [00:14<00:00, 68.04it/s]


In [32]:
dp.sort_values("prob")

,prob,text
439,56.578079,a photo of a person in the sea.
68,58.034519,a photo of a person with a guitar.
648,61.561375,a photo of a person walking through traffic.
487,61.679295,a photo of a person in another picture.
504,67.137917,a photo of a person with a debt.
...,...,...
376,1872.045166,a photo of a person constitution paper.
936,1957.345947,a photo of a person ferryeded.
293,2780.082520,a photo of a person cooper had from.
220,3535.363525,a photo of a person regional orized.


In [5]:
from domino.emb.clip import CELEBA_PHRASE_TEMPLATES
templates = CELEBA_PHRASE_TEMPLATES
words = word_dp["word"][:10]
input_phrases = [
    template.format(word) for word in words for template in templates
]
inputs = tokenizer(input_phrases, return_tensors="pt", padding=True).to(device)
input_ids = inputs["input_ids"]

outputs = model(**inputs) # shape=(num_sents, num_tokens_in_sent, size_vocab)

In [66]:
probs = torch.softmax(outputs.logits, dim=-1)

In [74]:
k = 2
top_k_out = probs.topk(k=k, dim=-1)
PAD_TOKEN_ID = 103

output_phrases = []
output_probs = []
for sent_idx in range(probs.shape[0]):
    mask_mask = input_ids[sent_idx] == PAD_TOKEN_ID
    token_ids = top_k_out.indices[sent_idx, mask_mask]
    token_probs = top_k_out.values[sent_idx, mask_mask]
    for local_idxs in product(np.arange(k), repeat=mask_mask.sum()):
        output_ids = torch.clone(input_ids[sent_idx])
        output_ids[mask_mask] = token_ids[torch.arange(mask_mask.sum()), local_idxs] 
        output_phrases.append(tokenizer.decode(output_ids, skip_special_tokens=True))
        output_probs.append(token_probs[torch.arange(mask_mask.sum()), local_idxs].mean().cpu().detach().numpy())



In [80]:
import pandas as pd
df = pd.DataFrame({"prob": output_probs, "phrase": output_phrases})
df.sort_values("prob", ascending=False)

,prob,phrase
0,nan,a photo of a person the.
454,0.49835655,a photo of a person dressed as a child.
442,0.49652216,a photo of a person as well.
455,0.48906392,a photo of a person dressed as a model.
475,0.3766427,a photo of a person dressed dressed as a person.
450,0.3659794,a photo of a person dressed as well.
441,nan,a photo of a person as.
401,0.37600285,a photo of a person sitting on it.
425,0.3701461,a photo of a person was placed on the wall.
429,0.36785626,a photo of a person was sitting on the wall.


In [ ]:
local_idxs

(1,)

In [49]:
from itertools import product
x[np.array(list(product(np.arange(k), repeat=4)))]

IndexError: too many indices for tensor of dimension 2

torch.return_types.topk(
values=tensor([[[4.8436e-02, 1.8082e-02],
         [9.9977e-01, 5.0052e-05],
         [6.9458e-01, 2.9586e-01],
         ...,
         [9.9806e-01, 1.2715e-03],
         [9.9830e-01, 1.1925e-03],
         [9.9728e-01, 2.0902e-03]],

        [[3.0408e-02, 1.0726e-02],
         [1.0000e+00, 6.8612e-08],
         [9.4834e-01, 5.1066e-02],
         ...,
         [9.7172e-01, 4.2474e-03],
         [3.1486e-01, 7.4373e-02],
         [1.6941e-01, 7.4333e-02]],

        [[2.8445e-02, 1.0210e-02],
         [1.0000e+00, 9.6760e-08],
         [9.6188e-01, 3.7719e-02],
         ...,
         [9.9834e-01, 8.7955e-04],
         [3.1594e-01, 6.6326e-02],
         [5.5921e-01, 1.2219e-01]],

        ...,

        [[2.3498e-02, 1.1045e-02],
         [1.0000e+00, 1.3866e-07],
         [9.6421e-01, 3.5359e-02],
         ...,
         [7.2595e-01, 2.3038e-01],
         [9.7169e-01, 5.1077e-03],
         [8.5109e-01, 4.2533e-02]],

        [[2.7845e-02, 8.7831e-03],
         [1.000